In [ ]:
!pip install tensorflow_hub

In [3]:
# Submission accuracy: 96.107%

# Import required libraries
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import regularizers

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint


# Define data directories
train_dir = '/common/scratch/projectgrps/CS424/assets/20230216/train_'


# Define batch size
batch_size = 32


# Define data generators
train_datagen = ImageDataGenerator(
    
    rescale=1./255,
    
    # Split the training images to 85% training, 15% testing
    validation_split=0.15,
    
    # Data Augmentation
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


# Define model – Transfer Learning based on BigTransfer (BiT) Model 
module_url = 'https://tfhub.dev/google/bit/m-r50x1/1'
module = hub.KerasLayer(module_url, trainable=True)
model = tf.keras.Sequential([
    module,
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=regularizers.L2(0.01)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu', kernel_regularizer=regularizers.L2(0.01)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu', kernel_regularizer=regularizers.L2(0.01)),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])


# Define Initial Learning Rate
learning_rate = 0.00005

# Define optimizer, loss function and metrics
model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define Callbacks
# callback 1 – Save best model during training
checkpoint_callback = ModelCheckpoint(filepath='bit_transfer_learning_model_4_BEST',
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      mode='max',
                                      verbose=1)
# callback 2 – Reduce the learning rate by factor of 10, with minimum of 5e-8
def lr_scheduler(epoch): 
    if epoch < 10:
        return learning_rate
    else:
        return max((learning_rate * 0.1 ** (epoch // 10)),5e-8)

                          
# Define epoch number
epoch_no = 40
        
# Train model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=epoch_no,
          validation_data=validation_generator,
          validation_steps=validation_generator.samples // batch_size,
          verbose=1,
          callbacks=[checkpoint_callback, LearningRateScheduler(lr_scheduler, verbose=1)],
          shuffle=True)


Found 8316 images belonging to 124 classes.
Found 1405 images belonging to 124 classes.

Epoch 1: LearningRateScheduler setting learning rate to 5e-05.
Epoch 1/40


2023-03-19 18:14:48.436467: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x82274360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-19 18:14:48.436512: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-03-19 18:14:48.442313: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-19 18:14:48.593740: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


259/259 [==============================] - ETA: 0s - loss: 13.6872 - accuracy: 0.6095
Epoch 1: val_accuracy improved from -inf to 0.90698, saving model to bit_transfer_learning_model_4_BEST
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(7, 7, 3, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f28c8ae9110>, 139815815563168), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(7, 7, 3, 64), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f28c8ae9110>, 139815815563168), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(64,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f28c8aa4850>, 139815294969392), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(2048,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f28abbf2d90>, 139813716732304), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(2048,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f28abbf2d90>, 139813716732304), {}).


INFO:tensorflow:Assets written to: bit_transfer_learning_model_4_BEST/assets


INFO:tensorflow:Assets written to: bit_transfer_learning_model_4_BEST/assets


259/259 [==============================] - 412s 2s/step - loss: 1.8384 - accuracy: 1.0000 - val_loss: 1.8970 - val_accuracy: 0.9891 - lr: 5.0000e-08

Epoch 34: LearningRateScheduler setting learning rate to 5.000000000000001e-08.
Epoch 34/40
259/259 [==============================] - ETA: 0s - loss: 1.8373 - accuracy: 0.9999
Epoch 34: val_accuracy did not improve from 0.98910
259/259 [==============================] - 394s 2s/step - loss: 1.8373 - accuracy: 0.9999 - val_loss: 1.8981 - val_accuracy: 0.9862 - lr: 5.0000e-08

Epoch 35: LearningRateScheduler setting learning rate to 5.000000000000001e-08.
Epoch 35/40
259/259 [==============================] - ETA: 0s - loss: 1.8358 - accuracy: 1.0000
Epoch 35: val_accuracy did not improve from 0.98910
259/259 [==============================] - 399s 2s/step - loss: 1.8358 - accuracy: 1.0000 - val_loss: 1.8906 - val_accuracy: 0.9855 - lr: 5.0000e-08

Epoch 36: LearningRateScheduler setting learning rate to 5.000000000000001e-08.
Epoch 36/40


In [20]:
import tensorflow as tf

# Load the best model 'bit_transfer_learning_model_4_BEST'
test_model = tf.keras.models.load_model('bit_transfer_learning_model_4_BEST')

2023-03-24 23:00:05.087875: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 're_lu_48/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 2 outputs. Output shapes may be inaccurate.
2023-03-24 23:00:05.088054: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'global_average_pooling2d/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 4 outputs. Output shapes may be inaccurate.


In [21]:
import numpy as np

from tensorflow.keras.preprocessing import image

# Define number of test images
num_test_images = 2005

# Initialise the predictions array
predictions = np.array([])

# Define the threshold 
# – Images with probability under this threshold value are considered as noise
threshold = 0.26

for i in range(2005):
    img = image.load_img('/common/scratch/projectgrps/CS424/assets/20230216/test_/' + str(i).zfill(6) + '.jpg', target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.
    
    probs = test_model(img_array)
    
    max_prob = np.max(probs)
    
    if max_prob >= threshold:
        predicted_label = np.argmax(probs)
    else:
        predicted_label = 124
    
    predictions = np.append(predictions, predicted_label)

In [23]:
import pandas as pd

# Convert the dateframe into required .csv file
data_columns = ['id', 'category']

result_df = pd.DataFrame(columns=data_columns)

for i in range (len(predictions)):
    
    new_row = {'id': str(i).zfill(6), 'category': int(predictions[i])}
        
    result_df = result_df.append(new_row, ignore_index=True)
    
result_df.to_csv('result.csv', index=False)

In [25]:
# Check the output

result_df

,id,category
0,000000,94
1,000001,101
2,000002,115
3,000003,14
4,000004,25
...,...,...
2000,002000,102
2001,002001,0
2002,002002,75
2003,002003,55
